In [13]:
!pip install blosum
import blosum as bl
import utils
import 

# Blosum scoring given a mutation locus:
blosum library allows us to easily index the blosum score for two amino acids

In [128]:
blosum_matrix = bl.BLOSUM(62)
print(blosum_matrix["_W"])
print(blosum_matrix["WW"])
utils.convert_to_amino('TGG')

-inf
11.0


'W'

In [168]:
def get_blosum_scores(nucleotide_index: int, original_seq: str) -> dict:
    '''Function assumes that nucleotide index (mutation location) is 1 indexed (not zero indexed)
    Function works with stop codons by nature of the blosum library'''
    blosum_matrix = bl.BLOSUM(62)
    ci = nucleotide_index%3 #ci -> codon index
    codon = original_seq[(nucleotide_index - ci): (nucleotide_index - ci + 3)]
    codon_amino = utils.convert_to_amino(codon.upper())
    possible_nucleotides = ["A", "T", "G", "C"]
    blosum_dict = {}
    for item in possible_nucleotides:
        if is_start_codon(codon): #if we have decided to mutate a start codon, don't
            blosum_dict[item] = float('-inf') if original_seq[nucleotide_index].upper() != item else blosum_matrix[codon_amino.upper()*2]
        else:
            new_codon = (codon[0: ci - 1] + item + codon[ci:3] if ci > 0 else codon[0: ci - 1] + item + codon[ci:0]).upper()
            new_amino = utils.convert_to_amino(new_codon)
            blosum_dict[item] = blosum_matrix[new_amino + codon_amino.upper()]
    return blosum_dict


print(f'''Messing with a stop codon: {get_blosum_scores(1, "taagcgcaagg")}\n''')
print(f'''Tryptophan: {get_blosum_scores(1, "tgggcgcaagg")}\n''')
print(f'''Messing with a start codon: {get_blosum_scores(1, "atggcgcaagg")}\n''')

Messing with a stop codon: {'A': -inf, 'T': -inf, 'G': -inf, 'C': -inf}

Tryptophan: {'A': -3.0, 'T': 11.0, 'G': -2.0, 'C': -3.0}

Messing with a start codon: {'A': -inf, 'T': 5.0, 'G': -inf, 'C': -inf}



In [169]:
def is_start_or_stop_codon_amino(s: str) -> bool:
    return s == "_" or s.lower() == "x" or s.lower() == "m"

def is_start_or_stop_codon_nucleotides(s: str) -> bool:
    s = s.lower()
    return s == "taa" or s == "tag" or s == "tga" or s == 'atg'

def is_start_codon(s:str) -> bool:
    return s == 'atg'

# argparse

In [170]:
import argparse

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('fasta_filepath', type = list, help='list of filepaths to fasta files')
parser.add_argument('mutation_rate', type = float, help='mutation rate')
parser.add_argument('diversity_file', type = str, help='diversity file')
parser.add_argument('mutation_time', type = str, help='number of years to mutation')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h]
                             fasta_filepath mutation_rate diversity_file
                             mutation_time
ipykernel_launcher.py: error: the following arguments are required: mutation_rate, diversity_file, mutation_time


SystemExit: 2

/Users/essawifamily/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
def simpleMatch(a, b):
    return 1 if a == b else -1

def distanceMatch(a, b):
    return 0 if a == b else -1

def linearGap(n):
    return -1 * n

def alignmentScoreDP(s1, s2, gapPenalty, match):
    m = np.zeros((len(s1) + 1, len(s2) + 1))
    m[0, 0] = 0
    for i in range(1, len(s1) + 1):
        m[i, 0] = gapPenalty(i)
    for j in range(1, len(s2) + 1):
        m[0, j] = gapPenalty(j)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            m[i, j] = max(gapPenalty(1) + m[i, j - 1],  
                          gapPenalty(1) + m[i - 1, j],    
                          match(s1[i - 1], s2[j - 1]) + m[i - 1, j - 1]) 
    return m
    
def readAlignment(s1, s2, m, gapPenalty, match):
    i = len(s1)
    j = len(s2)
    s1a = ""
    s2a = "" 
    score = 0
    while i > 0 or j > 0:
        if i > 0 and j > 0 and m[i, j] == m[i - 1, j - 1] + match(s1[i - 1], s2[j - 1]):
            i = i - 1
            j = j - 1
            score += match(s1[i], s2[j])
            s1a = s1[i] + s1a
            if s1[i] == s2[j]:
                s2a = s2[j] + s2a
            else:
                s2a = s2[j].lower() + s2a
        elif i > 0 and m[i, j] == m[i - 1, j] + gapPenalty(1):
            i = i - 1
            score += gapPenalty(1)
            s1a = s1[i] + s1a
            s2a = '-' + s2a
        elif j > 0 and m[i, j] == m[i, j - 1] + gapPenalty(1):
            j = j - 1
            score += gapPenalty(1)
            s1a = '-' + s1a
            s2a = s2[j] + s2a
        else:
            assert False
    return (s1a, s2a, score)

def showAlignment(s1, s2, gapPenalty, match):
    m = alignmentScoreDP(s1, s2, gapPenalty, match)
    r = readAlignment(s1, s2, m, gapPenalty, match)
    print (r[0] + "\n" + r[1] + "\n" + str(r[2]))
    return (m, r)

def readAlignmentG(s1, s2, m, gapPenalty, match):
    i = len(s1)
    j = len(s2)
    s1a = ""
    s2a = ""
    score = 0
    while i > 0 or j > 0:
        if i > 0 and j > 0 and m[i, j] == m[i - 1, j - 1] + match(s1[i - 1], s2[j - 1]):
            i = i - 1
            j = j - 1
            s1a = s1[i] + s1a
            s2a = (s2[j] if s1[i] == s2[j] else s2[j].lower()) + s2a
            score += match(s1[i], s2[j])
        else:
            foundit = False
            for g in range(1, i + 1):
                if m[i, j] == m[i - g, j] + gapPenalty(g):
                    s1a = s1[i - g:i] + s1a
                    s2a = ('-' * g) + s2a
                    i = i - g
                    score += gapPenalty(g)
                    foundit = True
                    break
            if not foundit:
                for g in range(1, j + 1):
                    if m[i, j] == m[i, j - g] + gapPenalty(g):
                        s1a = ('-' * g) + s1a
                        s2a = s2[j - g:j] + s2a
                        j = j - g
                        score += gapPenalty(g)
                        foundit = True
                        break
            assert foundit
        return 
    return (s1a, s2a, score)

In [141]:
#NOTE: This function is probably NOT needed. It was made without understanding clear specifications of the task

def check_start_and_stop_codons(original: str, divergent: str, blosum_score: int) -> float:
    original_amino_seq = original.lower()
    divergent_amino_seq = divergent.lower()
    for i in range(len(original_amino_seq)):
        if is_start_or_stop_codon(original_amino_seq[i]) or is_start_or_stop_codon(divergent_amino_seq[i]):
            if is_start_or_stop_codon(original_amino_seq[i]) != is_start_or_stop_codon(divergent_amino_seq[i]):
                return blosum_score - 100000000 #Give a really low blosum score
    return blosum_score
    
human_oca2, mouse_oca2 = utils.load_oca2_sequences()
human_oca2_amino = utils.convert_to_amino(human_oca2).lower()
mouse_oca2_amino = utils.convert_to_amino(mouse_oca2).lower()